In [54]:
import pdfplumber
import pandas as pd
import re

In [55]:
counties = ["Kosciusko", "Whitley", "Steuben"]

In [65]:
i = 0
county = counties[i]
county

'Kosciusko'

In [66]:
input_file = "../data-raw/in_2024_results/" + county + ".pdf"
output_file = "../data/in_2024_results/" + county + ".csv"
output_2 = "C:/Users/johnsor/Documents/openelections-data-in/2024/counties/20241105__in__general__" + str.lower(county) + "__precinct.csv"
output_2

'C:/Users/johnsor/Documents/openelections-data-in/2024/counties/20241105__in__general__kosciusko__precinct.csv'

In [67]:
with pdfplumber.open(input_file) as pdf:
    pages = [[line['text'] for line in page.extract_text_lines()] for page in pdf.pages]

In [68]:
page = pages[1]
page

['11/15/2024 - 12:35:07 PM',
 'Election Date: 11/5/2024 Precinct Summary Report',
 'EMS',
 'Kosciusko County, Indiana',
 '2024 General Election',
 'INKOSG24',
 '11/5/2024',
 'E - # Of Election Day 297 PRECINCT STATUS: INCOMPLETE',
 'A - # Of Paper Absentee 15',
 'W - # Of Walk-In Absentee 148',
 'PUBLIC COUNT: 460',
 'VOTER TURNOUT: 71.76%',
 'REGISTERED VOTERS: 641',
 'Precinct ID: 01',
 'Precinct Name: Clay 01',
 '- - - - - - - - - - - - - - - - - - VOTES - - - - - - - - - - - - - - - -',
 'E A W TOTAL %',
 'VOTE FOR 1',
 'Public Question-Const Amendment',
 'VOTES=403',
 '167 7 66 240 59.55% Yes',
 '103 2 58 163 40.45% No',
 'VOTE FOR 1',
 'President and Vice-President of the U.S.',
 'VOTES=458',
 '267 10 117 394 86.03% (R) Trump \\ Vance',
 '25 5 31 61 13.32% (D) Harris \\ Walz',
 '2 0 0 2 0.44% (L) Oliver \\ Ter Maat',
 '1 0 0 1 0.22% (WTP) Kennedy Jr. \\ Shanahan',
 '0 0 0 0 0.00% Write-In',
 'VOTE FOR 1',
 'United States Senator',
 'VOTES=438',
 '247 9 111 367 83.79% (R) Jim Bank

In [33]:
cols = {'county' : [],
        'precinct' : [],
        'office' : [],
        'district' : [],
        'party' : [],
        'candidate' : [],
        'election_day' : [],
        'absentee' : [],
        'early_voting' : [],
        'votes' : []}
party_convert = {'R' : 'REP',
                 'REP' : 'REP',
                 'D' : 'DEM',
                 'DEM' : 'DEM',
                 'L' : 'LIB',
                 'LIB' : 'LIB',
                 'WTP' : 'WTP',
                 'W' : 'WTP',
                 'NP' : ''
                }

In [34]:
for page in pages:
    vote = False #Signifies when a "VOTE FOR ..." line has occurred
    meta = False
    straight_party = False
    for (lineno, line) in enumerate(page): #for each line of the page
        if re.search('E - #', line):
            election_day = re.findall(r'Election Day (\d+) PRECINCT', line)[0]
            meta = True  #This is the first line of metadata
        elif re.search(r'^A - #', line):
            absentee = re.findall(r'Of Paper Absentee (\d+)', line)[0]
        elif re.search(r'^W - #', line):
            early_voting = re.findall(r'Walk-In Absentee (\d+)', line)[0]
        elif re.search(r'^PUBLIC COUNT', line):
            votes = re.findall(r'PUBLIC COUNT: (\d+)', line)[0]
        elif re.search(r'^REGISTERED VOTERS', line):
            registered = re.findall(r'REGISTERED VOTERS: (\d+)', line)[0]
        elif re.search(r'^Precinct Name', line) and meta:
            precinct = re.findall(r'Precinct Name: (.*)', line)[0]
            cols['county'].extend([county, county])
            cols['precinct'].extend([precinct, precinct])
            cols['office'].extend(['Ballots Cast', 'Registered Voters'])
            cols['district'].extend(['',''])
            cols['party'].extend(['',''])
            cols['candidate'].extend(['',''])
            cols['election_day'].extend([election_day,''])
            cols['absentee'].extend([absentee,''])
            cols['early_voting'].extend([early_voting,''])
            cols['votes'].extend([votes, registered])
        elif re.search('VOTE FOR', line):#The 'VOTE FOR..." line means the next line is what is being voted on
            vote = True
        elif vote: #If the 'VOTE FOR..." line occured earlier
            vote = False #Reset this
            meta = False #This line means we are not in meta data anymore
            if re.search(r'Senator District (\d+)|Representative District (\d+)', line):
                office, district = line.split(' District ')
            else:
                office = line
                district = ''
        elif re.search(r'\d+ \d+\.?\d*%', line) and not meta: #Lines with percentage numbers that are not metadata are the values we want
            E, A, W, V, _, option = line.split(maxsplit=5) #The line is a row of a table of 6 columns
            if re.search(r'\(\w+\)', option):
                P = party_convert[
                    re.findall(r'\((\w+)\)', option)[0]
                ]
                option = re.findall(r'\(\w+\) (.*)', option)[0]
            else:
                P = ''
            cols['county'].append(county)
            cols['precinct'].append(precinct)
            cols['office'].append(office)
            cols['district'].append(district)
            cols['party'].append(P)
            cols['candidate'].append(option)
            cols['election_day'].append(E)
            cols['absentee'].append(A)
            cols['early_voting'].append(W)
            cols['votes'].append(V)
        elif line == 'PartyName Votes':
            straight_party = True
        elif straight_party and re.search(r'\w+ \w+ \d+$', line):
            office = 'Straight Party'
            candidate = re.findall(r'(.*) \d+$', line)[0]
            V = re.findall(r'.* (\d+)$', line)[0]
            P = party_convert[line[0]]
            cols['county'].append(county)
            cols['precinct'].append(precinct)
            cols['office'].append(office)
            cols['district'].append(district)
            cols['party'].append(P)
            cols['candidate'].append(candidate)
            cols['election_day'].append('')
            cols['absentee'].append('')
            cols['early_voting'].append('')
            cols['votes'].append(V)
        else:
            straight_party = False
            

In [35]:
df = pd.DataFrame(data=cols)

In [37]:
df.to_csv(output_file)

In [38]:
df.to_csv(output_2, index=False)

### Double Check
The code below was used to double check our Whitley data vs the Whitley data that they had already done.  As of 1/28/2025 it checks out.

In [129]:
output_3 = "C:/Users/johnsor/Documents/openelections-data-in/2024/counties/20241105__in__general__" + str.lower(county) + "__precinct_1.csv"

In [130]:
df.to_csv(output_3, index=False)

In [131]:
df1 = pd.read_csv(output_3)
df2 = pd.read_csv(output_2)

In [133]:
pd.concat([df1,df2]).drop_duplicates(keep=False).sort_values(by=['precinct', 'office', 'candidate'])

,county,precinct,office,district,party,candidate,election_day,absentee,early_voting,votes


Since the non-duplicates of the two concatenated dataframes is empty, they are both exactly the same

In [64]:
pdf.pages[0].extract_text_lines()

[]